# Project Pauli operators onto Hilbert subspaces

In [2]:
import numpy as np
from qiskit_addon_sqd.qubit import matrix_elements_from_pauli_string, sort_and_remove_duplicates

L = 22

# Write all of the Pauli strings for the Heisenberg model
paulis = []
for i in range(L):
    pstr = ["I" for i in range(L)]
    # Sigma_x
    pstr[i] = "X"
    pstr[(i + 1) % L] = "X"
    paulis.append(pstr)

    pstr = ["I" for i in range(L)]
    # Sigma_y
    pstr[i] = "Y"
    pstr[(i + 1) % L] = "Y"
    paulis.append(pstr)

    pstr = ["I" for i in range(L)]
    # Sigma_z
    pstr[i] = "Z"
    pstr[(i + 1) % L] = "Z"
    paulis.append(pstr)

Let's make some random bitstrings

In [3]:
rand_seed = 22
np.random.seed(rand_seed)


def random_bitstrings(n_samples, n_qubits):
    return np.round(np.random.rand(n_samples, n_qubits)).astype("int").astype("bool")


bts_matrix = random_bitstrings(50_000_000, L)

# NOTE: It is essential for the projection code to have the bitstrings sorted!
bts_matrix = sort_and_remove_duplicates(bts_matrix)

print("Subspace dimension: " + str(bts_matrix.shape[0]))
print("Full Hilbert space dimension: " + str(2**L))

Subspace dimension: 4194276
Full Hilbert space dimension: 4194304


In [4]:
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import eigsh

d = bts_matrix.shape[0]

# The first Pauli operator
matrix_elements, row_coords, col_coords = matrix_elements_from_pauli_string(bts_matrix, paulis[0])

# The complex double precision is required to match exactly Netket's results
# We can relax it to complex64 most likely
ham = coo_matrix((matrix_elements, (row_coords, col_coords)), (d, d), dtype="complex128")

# The remaining Pauli operators
# It will be a good idea to make this operation in parallel
for i in range(len(paulis) - 1):
    matrix_elements, row_coords, col_coords = matrix_elements_from_pauli_string(
        bts_matrix, paulis[i + 1]
    )
    ham += coo_matrix((matrix_elements, (row_coords, col_coords)), (d, d))

In [5]:
# And we finally diagonalize
E, V = eigsh(ham, k=1, which="SA")

print(E)

[-39.14735935]
